In [122]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
from collections import deque
from torch.utils.data import DataLoader

In [7]:
def rewards_to_go(rewards, discount_factor=0.99):
    # from https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html
    
    r2g = []
    discounted_reward = 0
    
    for reward in reversed(rewards):
        discounted_reward = reward + discount_factor * discounted_reward
        r2g.insert(0, discounted_reward)
    
    return torch.tensor(r2g, dtype=torch.float)

In [407]:
def compute_advantage(rewards, state_values):
    advantages = rewards - state_values
    return normalize(advantages)
    #return advantages

In [422]:
def normalize(x):
    if torch.isnan(x.std()):
        return x - x.mean(0)
    
    return (x - x.mean(0)) / (x.std(0) + 1e-8)

In [423]:
class Trajectory(torch.utils.data.Dataset):
    def __init__(self):
        self.states = []
        self.log_probs = []
        self.actions = []
        self.rewards = []
        self.dones = []

    def __len__(self):
        return len(self.states)
    
    def convert_rewards_to_go(self, discount_factor=0.99):
        # from https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html
        r2g = []
        discounted_reward = 0

        for reward in reversed(self.rewards):
            discounted_reward = reward + discount_factor * discounted_reward
            r2g.insert(0, discounted_reward)

        self.rewards = torch.tensor(r2g, dtype=torch.float)
        
    def fix_datatypes(self):
        self.states = torch.stack(self.states)
        self.actions = torch.tensor(self.actions, dtype=torch.long)
        #self.rewards = torch.tensor(self.rewards, dtype=torch.float)
        self.dones = torch.tensor(self.dones, dtype=torch.int)
        self.log_probs = torch.tensor(self.log_probs, dtype=torch.float)
        

    def store_timestep(self, state, action, reward, done, log_prob):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.dones.append(done)
        self.log_probs.append(log_prob)
        
    def clear_memory(self):
        self.states = []
        self.log_probs = []
        self.actions = []
        self.rewards = []
        self.dones = []
    
    def __getitem__(self, index):
        state = self.states[index]
        action = self.actions[index]
        reward = self.rewards[index]
        done = self.dones[index]
        log_prob = self.log_probs[index]
        
        return state, action, reward, done, log_prob

In [430]:
class Agent:
    def __init__(self, env, action_dim, state_dim, batch_size, clip_ratio=0.2):
        self.env = env
        self.actor = Policy(action_dim, state_dim)
        self.critic = CriticNet(state_dim)
        self.trajectories = deque(maxlen=5)
        self.discount_factor = 0.99
        self.batch_size = batch_size
        self.clip_ratio = clip_ratio
        self.actor_opt = optim.Adam(self.actor.parameters(), lr=1e-3)
        self.critic_opt = optim.Adam(self.critic.parameters(), lr=1e-3)
        
    def get_action(self, obs):
        action_probs = self.actor(obs)
        #print(obs)
        #print(action_probs)
        if torch.isnan(action_probs).any():
            print(obs)
            print(action_probs)
        action_dist = Categorical(logits=action_probs)
        action = action_dist.sample()
        
        return action.item(), action_dist.log_prob(action).item()
    
    def train(self):
        for trajectory in self.trajectories:
            loader = DataLoader(trajectory, batch_size=self.batch_size, shuffle=False)
            
            for states, actions, rewards, dones, old_log_probs in loader:
                state_values = self.critic(states)
                state_values = state_values.squeeze()
                advantages = compute_advantage(rewards, state_values.detach())
                
                action_probs = self.actor(states)
                #print('ACTION PROBS', action_probs)
                action_dist = Categorical(logits=action_probs)
                log_probs = action_dist.log_prob(actions)
                
                # log trick for efficient computational graph during backprop
                ratio = torch.exp(log_probs - old_log_probs)
                # probs = torch.exp(log_probs) / torch.exp(old_log_probs)
                
                weighted_objective = advantages * ratio
                #print('ADVA', advantages)
                #print('ratio:', ratio)
                clipped_objective = torch.clamp(ratio, 1 - self.clip_ratio, 1 + self.clip_ratio) * advantages
                
                objective = -torch.min(weighted_objective, clipped_objective).mean()
                self.actor_opt.zero_grad()
                objective.backward(retain_graph=True)
                self.actor_opt.step()
                
                self.critic_opt.zero_grad()
                loss = nn.MSELoss()(state_values, rewards)
                loss.backward()
                self.critic_opt.step()
        
    
    def run_episode(self, render=False):
        trajectory = Trajectory()
        
        done = False
        obs = self.env.reset()
        
        while not done:
            if render:
                self.env.render()

            obs = torch.from_numpy(obs).float()
            action, log_prob = agent.get_action(obs)
            next_obs, reward, done, _ = self.env.step(action)
            
            trajectory.store_timestep(obs, action, reward, done, log_prob)
            obs = next_obs
            
        trajectory.convert_rewards_to_go()
        trajectory.fix_datatypes()
        self.trajectories.append(trajectory)
        

In [431]:
class Policy(nn.Module):
    def __init__(self, action_dim, state_dim):
        super(Policy, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(state_dim, 100),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(100, action_dim),
            nn.Softmax(dim=-1)
        )
        
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [432]:
class CriticNet(nn.Module):
    def __init__(self, state_dim):
        super(CriticNet, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(state_dim, 100),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(100, 1)
        )       
        
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [433]:
env = gym.make("CartPole-v0")
agent = Agent(env, 2, 4, 16)

In [434]:
for _ in range(5):
    agent.run_episode()
    
for _ in range(105):
    agent.train()
    agent.run_episode()